In [1]:
import wandb
import numpy as np
import os
from activations import Sigmoid, Tanh, Relu, Softmax
from layers import Input, Dense
from optimizers import Momentum, Nesterov, AdaGrad, RMSProp, Adam, Nadam
from network import NeuralNetwork
from loss import CrossEntropy
from helper import OneHotEncoder, MinMaxScaler

from sklearn.model_selection import train_test_split

from keras.datasets import fashion_mnist
import matplotlib.pyplot as plt

# Loss on Training Data

In [4]:
# Preparing small dataset to test the code
[(X_train, y_train), (X_test, y_test)] = fashion_mnist.load_data()
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_train)
X_scaled = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1]*X_scaled.shape[2]).T

encoder = OneHotEncoder()
t = encoder.fit_transform(y_train, 10)

In [5]:
sweep_config = {'name': 'random-test-sweep', 'method': 'grid'}
sweep_config['metric'] = {'name': 'loss', 'goal': 'minimize'}
parameters_dict = {
                   'num_epochs': {'values': [5]}, \
                   #'num_hidden_layers': {'values': [3, 4, 5]}, \
                   'size_hidden_layer': {'values': [32, 64, 128]}, \
                   #'learning_rate': {'values': [1e-3, 1e-4]}, \
                   'optimizer': {'values': ["Adam"]}, \
                   'batch_size': {'values': [60000]}, \
                   'weight_init': {'values': ['RandomNormal', 'XavierUniform']} , \
                   #'activation': {'values': {'sigmoid', 'tanh'}}, \
                  }
sweep_config['parameters'] = parameters_dict
print(sweep_config)

{'name': 'random-test-sweep', 'method': 'grid', 'metric': {'name': 'loss', 'goal': 'minimize'}, 'parameters': {'num_epochs': {'values': [5]}, 'size_hidden_layer': {'values': [32, 64, 128]}, 'optimizer': {'values': ['Adam']}, 'batch_size': {'values': [60000]}, 'weight_init': {'values': ['RandomNormal', 'XavierUniform']}}}


In [6]:
def train_nn(config = sweep_config):
    with wandb.init(config = config):
        config = wandb.init().config
        wandb.run.name = 'epochs_{}layersize_{}opt_{}batch_{}init_{}'.format(config.num_epochs, config.size_hidden_layer, \
                          config.optimizer, config.batch_size, config.weight_init)
        layers = [Input(data = X_scaled), Dense(size = config.size_hidden_layer, activation = "Sigmoid", name = "HL1"), \
                  Dense(size = 10, activation = "Sigmoid", name = "OL")]
        nn_model = NeuralNetwork(layers = layers, batch_size = config.batch_size, optimizer = config.optimizer, \
                                 intialization = config.weight_init, epochs = config.num_epochs, t = t)
        nn_model.forward_propogation()
        nn_model.backward_propogation()
        
        for step_loss in nn_model.loss_hist:
            wandb.log({'loss': step_loss})

In [7]:
sweep_id = wandb.sweep(sweep_config, project = 'Train-NN-13')

Create sweep with ID: lt452qcu
Sweep URL: https://wandb.ai/shubham_kashyapi/Train-NN-13/sweeps/lt452qcu


In [8]:
wandb.agent(sweep_id, function = train_nn)

wandb: Agent Starting Run: epg3woit with config:
wandb: 	batch_size: 60000
wandb: 	num_epochs: 5
wandb: 	optimizer: Adam
wandb: 	size_hidden_layer: 32
wandb: 	weight_init: RandomNormal
wandb: Currently logged in as: shubham_kashyapi (use `wandb login --relogin` to force relogin)


wandb: ERROR Run epg3woit errored: AttributeError("'Dense' object has no attribute 'W'")


(784, 60000)


wandb: Agent Starting Run: 9b470oew with config:
wandb: 	batch_size: 60000
wandb: 	num_epochs: 5
wandb: 	optimizer: Adam
wandb: 	size_hidden_layer: 32
wandb: 	weight_init: XavierUniform


(784, 60000)


...

100%|██████████| 5/5 [00:02<00:00,  1.68it/s]
wandb: Agent Starting Run: 29rk2nhb with config:
wandb: 	batch_size: 60000
wandb: 	num_epochs: 5
wandb: 	optimizer: Adam
wandb: 	size_hidden_layer: 128
wandb: 	weight_init: RandomNormal


(784, 60000)


wandb: ERROR Run 29rk2nhb errored: AttributeError("'Dense' object has no attribute 'W'")
wandb: Agent Starting Run: c8lc4www with config:
wandb: 	batch_size: 60000
wandb: 	num_epochs: 5
wandb: 	optimizer: Adam
wandb: 	size_hidden_layer: 128
wandb: 	weight_init: XavierUniform


(784, 60000)


100%|██████████| 5/5 [00:04<00:00,  1.02it/s]
wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


# Loss on Validation Data

In [2]:
# Preparing small dataset to test the code
[(X_train, y_train), (X_test, y_test)] = fashion_mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1]*X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1]*X_test.shape[2])
Xtrain_scaled = StandardScaler().fit_transform(X_train)
Xtest_scaled = StandardScaler().fit_transform(X_test)

t_train = OneHotEncoder().fit_transform(y_train.reshape(-1,1))
t_test = OneHotEncoder().fit_transform(y_test.reshape(-1,1))
print(Xtrain_scaled.shape, Xtest_scaled.shape, t_train.shape, t_test.shape)

(60000, 784) (10000, 784) (60000, 10) (10000, 10)


In [3]:
# Splitting into train and validation sets
X_model, X_val, t_model, t_val = train_test_split(Xtrain_scaled, t_train, test_size=0.1, random_state=50)
X_model, X_val = X_model.T, X_val.T
t_model, t_val = t_model.T, t_val.T
print(X_model.shape, t_model.shape, X_val.shape, t_val.shape)

(784, 54000) (10, 54000) (784, 6000) (10, 6000)


In [13]:
sweep_config = {'name': 'random-test-sweep', 'method': 'grid'}
sweep_config['metric'] = {'name': 'loss', 'goal': 'minimize'}
parameters_dict = {
                   'num_epochs': {'values': [5]}, \
                   #'num_hidden_layers': {'values': [3, 4, 5]}, \
                   'size_hidden_layer': {'values': [32, 64, 128]}, \
                   #'learning_rate': {'values': [1e-3, 1e-4]}, \
                   'optimizer': {'values': ["Adam"]}, \
                   'batch_size': {'values': [54000]}, \
                   'weight_init': {'values': ['XavierUniform']} 
                   #'activation': {'values': {'sigmoid', 'tanh', 'relu'}}, \
                  }
sweep_config['parameters'] = parameters_dict
print(sweep_config)

{'name': 'random-test-sweep', 'method': 'grid', 'metric': {'name': 'loss', 'goal': 'minimize'}, 'parameters': {'num_epochs': {'values': [5]}, 'size_hidden_layer': {'values': [32, 64, 128]}, 'optimizer': {'values': ['Adam']}, 'batch_size': {'values': [54000]}, 'weight_init': {'values': ['XavierUniform']}}}


In [14]:
def train_val_nn(config = sweep_config):
    with wandb.init(config = config):
        config = wandb.init().config
        wandb.run.name = 'epochs_{}layersize_{}opt_{}batch_{}init_{}'.format(config.num_epochs, config.size_hidden_layer, \
                          config.optimizer, config.batch_size, config.weight_init)
        layers = [Input(data = X_model), Dense(size = config.size_hidden_layer, activation = "Sigmoid", name = "HL1"), \
                  Dense(size = 10, activation = "Sigmoid", name = "OL")]
        nn_model = NeuralNetwork(layers = layers, batch_size = config.batch_size, optimizer = config.optimizer, \
                              intialization = config.weight_init, epochs = config.num_epochs, t = t_model)
        nn_model.forward_propogation()
        nn_model.backward_propogation()
        
        for step_loss in nn_model.loss_hist:
            wandb.log({'loss': step_loss})

In [15]:
sweep_id = wandb.sweep(sweep_config, project = 'Train-NN-3')

Create sweep with ID: kl1mg7vo
Sweep URL: https://wandb.ai/shubham_kashyapi/Train-NN-3/sweeps/kl1mg7vo


In [16]:
wandb.agent(sweep_id, function = train_val_nn)

wandb: Agent Starting Run: tm0cz4me with config:
wandb: 	batch_size: 54000
wandb: 	num_epochs: 5
wandb: 	optimizer: Adam
wandb: 	size_hidden_layer: 32
wandb: 	weight_init: XavierUniform


(784, 54000)


  0%|          | 0/5 [00:00<?, ?it/s]
wandb: ERROR Run tm0cz4me errored: ValueError('dimension mismatch')
wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: ykzpl3mu with config:
wandb: 	batch_size: 54000
wandb: 	num_epochs: 5
wandb: 	optimizer: Adam
wandb: 	size_hidden_layer: 64
wandb: 	weight_init: XavierUniform


(784, 54000)


  0%|          | 0/5 [00:00<?, ?it/s]
wandb: ERROR Run ykzpl3mu errored: ValueError('dimension mismatch')
wandb: Agent Starting Run: clfc8xpg with config:
wandb: 	batch_size: 54000
wandb: 	num_epochs: 5
wandb: 	optimizer: Adam
wandb: 	size_hidden_layer: 128
wandb: 	weight_init: XavierUniform


(784, 54000)


  0%|          | 0/5 [00:00<?, ?it/s]
wandb: ERROR Run clfc8xpg errored: ValueError('dimension mismatch')
wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
